In [1]:
from manim import *
import jupyter_capture_output

video_scene = " -v WARNING  --disable_caching sswave_Scene"
image_scene = f" -v WARNING --disable_caching -r {2*427},{2*240}  -s sswave_Scene"

Jupyter Capture Output v0.0.11


In [106]:
class SSWave(Mobject):
	def __init__(self, center, height, width, baseline = 0, c = 1, lam = 1, alpha = PI / 4, **kwargs):
		super().__init__(**kwargs)
		
		self.rect_center = center
		self.rect_height = height
		self.rect_width = width
		self.baseline = baseline

		self.c = c
		self.lam = lam
		self.omega = 2*PI * c / lam
		self.k = self.omega / c 
		self.alpha = alpha

		# setting the coordinates
		self.x_min = -width / 2
		self.x_max = width / 2
		self.y_min = 0
		self.y_max = height

		sswave_rectangle = Rectangle(height = self.rect_height, width = self.rect_width, color = WHITE, fill_color = GRAY, fill_opacity = 0).move_to(self.rect_center)
		self.add(sswave_rectangle)


	def __get_inc_line_from_pos(self, pos):
		x_down = pos[0] + (self.y_min - pos[1]) * np.tan(self.alpha) 
		x_up = pos[0] + (self.y_max - pos[1]) * np.tan(self.alpha) 
		y_left = pos[1] + (self.x_min - pos[0]) / np.tan(self.alpha)
		y_right = pos[1] + (self.x_max - pos[0]) / np.tan(self.alpha)

		if (x_down < self.x_min and y_left > self.y_max) or (x_up > self.x_max and y_right < self.y_min):
			return (np.array([-99, -99, -99]), np.array([-99, -99, -99]))
		else: 
			inc_start = np.array([max(x_down, self.x_min), max(y_left, self.y_min), 0]) + self.rect_center + DOWN*self.rect_height/2
			inc_end = np.array([min(x_up, self.x_max), min(y_right, self.y_max), 0]) + self.rect_center + DOWN*self.rect_height/2
			return (inc_start, inc_end)
	

	def __get_refl_line_from_pos(self, pos):
		x_down = pos[0] - (self.y_min - pos[1]) * np.tan(self.alpha) 
		x_up = pos[0] - (self.y_max - pos[1]) * np.tan(self.alpha) 
		y_left = pos[1] - (self.x_min - pos[0]) / np.tan(self.alpha)
		y_right = pos[1] - (self.x_max - pos[0]) / np.tan(self.alpha)

		if (x_down < self.x_min and y_left < self.y_min) or (x_up > self.x_max and y_right > self.y_max):
			return (np.array([-99, -99, -99]), np.array([-99, -99, -99]))
		else:
			refl_start = np.array([max(x_up, self.x_min), min(y_left, self.y_max), 0]) + self.rect_center + DOWN*self.rect_height/2
			refl_end = np.array([min(x_down, self.x_max), max(y_right, self.y_min), 0]) + self.rect_center + DOWN*self.rect_height/2
		return (refl_start, refl_end)


	def __get_inc_line(self, pos, amplitude):
		inc_line_start, inc_line_end = self.__get_inc_line_from_pos(pos)
		if inc_line_start[0] == -99:
			return False
		else:
			# bg_line = Line(start = inc_line_start, end = inc_line_end, color = BLACK, stroke_opacity = 0.5)
			if amplitude > 0:
				line = Line(start = inc_line_start, end = inc_line_end, color = WHITE, stroke_opacity = amplitude)
			else:
				line = Line(start = inc_line_start, end = inc_line_end, color = BLACK, stroke_opacity = np.abs(amplitude))
			return VGroup(line)
	

	def __get_refl_line(self, pos, amplitude):
		refl_line_start, refl_line_end = self.__get_refl_line_from_pos(pos)
		if refl_line_start[0] == -99:
			return False
		else:
			# bg_line = Line(start = refl_line_start, end = refl_line_end, color = BLACK, stroke_opacity = 0.5)
			if amplitude > 0:
				line = Line(start = refl_line_start, end = refl_line_end, color = WHITE, stroke_opacity = amplitude)
			else:
				line = Line(start = refl_line_start, end = refl_line_end, color = BLACK, stroke_opacity = np.abs(amplitude))
			return VGroup(line)
	

	def __get_y_inc(self, x):
		return -np.tan(self.alpha) * x
	

	def __get_y_refl(self, x):
		return np.tan(self.alpha) * x
	

	def get_phi_inc(self, pos, t):
		return 0.5 * np.cos(self.k * (np.cos(self.alpha)*pos[0] - np.sin(self.alpha)*pos[1]) - self.omega*t)
	

	def get_phi_refl(self, pos, t):
		return 0.5 * np.cos(self.k * (np.cos(self.alpha)*pos[0] + np.sin(self.alpha)*pos[1]) - self.omega*t)
	

	def get_inc_wavefield(self, x_range, t):
		x1 = x_range[0]
		x = x_range[1]
		inc_wave_group = VGroup()
		while x > x1:
			y = self.__get_y_inc(x)
			inc_pos = (x, y)
			inc_wave_amplitude = self.get_phi_inc(inc_pos, t)
			inc_wave_line = self.__get_inc_line(inc_pos, inc_wave_amplitude)
			if inc_wave_line:
				inc_wave_group.add(inc_wave_line)
			x -= 0.02
		return inc_wave_group
	

	def get_refl_wavefield(self, x_range, t):
		x1 = x_range[0]
		x = x_range[1]
		refl_wave_group = VGroup()
		while x > x1:
			y = self.__get_y_refl(x)
			refl_pos = (x, y)
			refl_wave_amplitude = self.get_phi_refl(refl_pos, t)
			refl_wave_line = self.__get_refl_line(refl_pos, refl_wave_amplitude)
			if refl_wave_line:
				refl_wave_group.add(refl_wave_line)
			x -= 0.02
		return refl_wave_group
	


	def get_wavefield(self, t):
		x1 = - (self.height / np.tan(self.alpha) + self.width) / 2
		x2 = x1 + self.c * np.cos(self.alpha) * t
		inc_wavefield = self.get_inc_wavefield([x1, x2], t)
		wavefield = VGroup(inc_wavefield)
		if x2 > -self.width/4:
			refl_wavefield = self.get_refl_wavefield([x1, x2], t)
			wavefield.add(refl_wavefield)
		return wavefield

In [107]:
%%manim -qh --fps 60 $video_scene


class sswave_Scene(Scene):
	def construct(self):
		CVC = Text('CVC', font_size = 12, weight = BOLD, color = WHITE, font = 'Latin Modern Sans').align_on_border(RIGHT + DOWN, buff = 0.2)
		self.add(CVC)

		# headline
		headline = Title(r"Stehende Skalare Welle", font_size = 48).align_on_border(UP + LEFT, buff = 0.5).shift(0.5 * RIGHT)
		self.add(headline)

		
		# parameters
		center = np.array([0, -0.25, 0])
		height = 5
		width = 12


		# physical parameters
		c = 1
		lam = 1
		alpha = PI / 4					# 45 Grad

		sswave = SSWave(center, height, width)
		self.add(sswave)

		sswave_wavefield = sswave.get_wavefield(0)
		self.add(sswave_wavefield)


		def wavefield_updater(field):
			t = t_tracker.get_value()
			new_wavefield = sswave.get_wavefield(t)
			field.become(new_wavefield)


		# self.wait(1)
		t_tracker = ValueTracker(0)
		sswave_wavefield.add_updater(wavefield_updater)
		self.play(t_tracker.animate.set_value(20), rate_func = linear, run_time = 20)
		# self.wait(3)

Manim Community v0.18.1